In [1]:
import os
import numpy as np
import pywt
#from tqdm import tqdm

c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# 加载数据
datafilename1 = "cinc2017Seg.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [3]:
import numpy as np
from scipy.fft import fft, ifft
def autocorrelation(signal):
    N = len(signal)
    signal -= np.mean(signal)  
    PSD = np.abs(fft(signal))**2
    autocorr = ifft(PSD)
    autocorr = np.real(autocorr)  
    autocorr /= autocorr[0]  
    return autocorr[:N]
def nzopp(signal):
    acf = autocorrelation(signal)
    return np.sum(np.abs(np.diff(np.sign(acf))) > 0) / len(acf)
def find_optimal_threshold(data, wavelet='db7', level=9, initial_thresholds=[0.1, 0.5, 0.9]):
    optimal_threshold = None
    threshold_diff = 1
    thresholds = np.array(initial_thresholds)
    while threshold_diff > 1e-10:
        r_values = []
        for threshold in (thresholds):
            coeffs = pywt.wavedec(data=data, wavelet=wavelet, level=level)
            for i in range(1, len(coeffs)):
                coeffs[i] = pywt.threshold(coeffs[i], threshold, mode='soft')
            rdata = pywt.waverec(coeffs=coeffs, wavelet=wavelet)
            r_values.append(nzopp(rdata))
        max_index = np.argmax(r_values)
        optimal_threshold = thresholds[max_index]
        if max_index == 0:
            new_thresholds = np.linspace(thresholds[0], thresholds[1], 5)
        elif max_index == len(thresholds) - 1:
            new_thresholds = np.linspace(thresholds[-2], thresholds[-1], 5)
        else:
            new_thresholds = np.linspace(thresholds[max_index-1], thresholds[max_index+1], 5)
        threshold_diff = np.max(np.abs(np.diff(new_thresholds)))
        thresholds = new_thresholds
    return optimal_threshold
def denoise(data, wavelet='db7', level=9):
    optimal_threshold = find_optimal_threshold(data, wavelet=wavelet, level=level)
    coeffs = pywt.wavedec(data=data, wavelet=wavelet, level=level)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], optimal_threshold, mode='soft')
    rdata = pywt.waverec(coeffs=coeffs, wavelet=wavelet)
    return rdata

In [4]:
X_val_denoise = [denoise(signal) for signal in (X_val)]

c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 9 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [5]:
X_test_denoise = [denoise(signal) for signal in (X_test)]

In [6]:
X_train_denoise = [denoise(signal) for signal in (X_train)]

In [8]:
X_train_denoise_matrix = np.array(X_train_denoise)
X_val_denoise_matrix = np.array(X_val_denoise)
X_test_denoise_matrix = np.array(X_test_denoise)

In [9]:
X_train_denoise = X_train_denoise_matrix.astype(np.int16)
X_val_denoise = X_val_denoise_matrix .astype(np.int16)
X_test_denoise = X_test_denoise_matrix.astype(np.int16)

In [ ]:
np.savez('cinc2017denoise',ecgstrain=X_train_denoise_matrix,labelstrain= y_train,ecgstest=X_test_denoise_matrix,labelstest= y_test,ecgsval=X_val_denoise_matrix,labelsval= y_val)